# MONTAR O DRIVE

In [34]:
# MONTAR O DRIVE

# Importar utilitário de montagem de drive (funciona no Colab e localmente)
from importlib.util import spec_from_file_location, module_from_spec
import os

# Localizar o arquivo utils_drive.py
utils_drive_path = os.path.join(os.getcwd(), '02_src', 'utils_drive.py')
if not os.path.exists(utils_drive_path):
    # Tentar caminho alternativo se estiver em subdiretório
    utils_drive_path = os.path.join(os.path.dirname(os.getcwd()), '02_src', 'utils_drive.py')

# Importar o módulo dinamicamente
spec = spec_from_file_location('utils_drive', utils_drive_path)
utils_drive = module_from_spec(spec)
spec.loader.exec_module(utils_drive)

# Configurar ambiente (monta drive e retorna caminhos)
drive_path, project_root = utils_drive.setup_environment()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# GERAÇÃO DA BASE

# VALIDAÇÃO

---
# TROCA DE ESTRATEGISTA - RETORNO AO SILVER PARA CORREÇÃO

In [42]:
# INSTRUÇÃO #0002 — SILVER_REPAIR_AND_HARDEN_V2 (DRY RUN)
# Objetivo: Regenerar em RAM o silver_close_20250923_v2.parquet a partir do Bronze
#           e adicionar coluna sha256 ao manifesto (em RAM, sem persistir).
# Método de calendário: união explícita dos índices dos 24 tickers .SA (Bronze)

import os
import io
import re
import sys
import json
import hashlib
from datetime import datetime
import pandas as pd
import numpy as np

# =============================== CONFIG ===============================
DRY_RUN = True  # obrigatório
EXPECTED_ROWS = 3409
EXPECTED_COLS = 31
EXPECTED_DATE_MIN = "2012-01-02"
EXPECTED_DATE_MAX = "2025-09-19"
EXPECTED_ARTIFACT_NAME = "silver_close_20250923_v2.parquet"

MANIFEST_CSV = "/content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/02_silver/silver_manifest_20250923_v2.csv"

# Derivar ROOT e RAW_DIR a partir do caminho do manifesto (sem adivinhar nomes externos).
# Resultado esperado: /content/drive/Shareddrives/BOLSA_2026/a_bolsa2026_gemini/00_data/01_raw/
proj_root = os.path.dirname(os.path.dirname(MANIFEST_CSV))  # .../a_bolsa2026_gemini/00_data
RAW_DIR = os.path.join(proj_root, "01_raw")

# =========================== GUARD-CLAUSES ============================
def fail(msg, payload=None):
    print("\n=== VALIDATION_ERROR ===")
    print(msg)
    if payload is not None:
        try:
            print(json.dumps(payload, indent=2, ensure_ascii=False, default=str))
        except Exception:
            print(payload)
    sys.exit(1)

if not os.path.exists(MANIFEST_CSV):
    fail("Manifesto não encontrado no SSOT.", {"MANIFEST_CSV": MANIFEST_CSV})

if not os.path.isdir(RAW_DIR):
    fail("Diretório RAW ausente no SSOT.", {"RAW_DIR": RAW_DIR})

# ============================== UTIL ==============================
def read_parquet_close_series(fpath):
    """Lê parquet, detecta coluna Close (case-insensitive) e retorna Series com DatetimeIndex canônico."""
    try:
        df = pd.read_parquet(fpath)
    except Exception as e:
        fail("Falha ao ler Parquet do Bronze.", {"arquivo": fpath, "erro": repr(e)})

    # Detectar coluna de data (preferência: column; fallback: index)
    date_col = None
    for cand in ("date", "Date", "DATE"):
        if cand in df.columns:
            date_col = cand
            break
    if date_col is None:
        if isinstance(df.index, pd.DatetimeIndex):
            df = df.copy()
            df["__date__"] = df.index
            date_col = "__date__"
        else:
            fail("Nenhum índice/coluna de data detectado no Bronze.", {"arquivo": fpath, "index_type": type(df.index).__name__})

    # Detectar coluna Close (case-insensitive)
    cols_lower = {c.lower(): c for c in df.columns}
    close_col = cols_lower.get("close")
    if close_col is None:
        fail("Coluna 'Close' ausente no artefato do Bronze.", {"arquivo": fpath, "colunas": list(df.columns)})

    # Normalizar datas
    s = df[[date_col, close_col]].copy()
    s[date_col] = pd.to_datetime(s[date_col], utc=False, errors="coerce")
    if s[date_col].isna().any():
        fail("Datas inválidas encontradas no Bronze.", {"arquivo": fpath, "n_nulos": int(s[date_col].isna().sum())})
    s = s.set_index(date_col).sort_index()
    if isinstance(s.index, pd.DatetimeIndex) and s.index.tz is not None:
        s.index = s.index.tz_localize(None)

    # Nome da coluna derivado do arquivo (sem extensão), minúsculo
    col_name = os.path.splitext(os.path.basename(fpath))[0].lower()
    return col_name, s[close_col].rename(col_name)

# ============================== PASSO 1 ===============================
# Listar todos os arquivos parquet no RAW
parquet_files = sorted([os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.lower().endswith(".parquet")])
if len(parquet_files) == 0:
    fail("Nenhum arquivo .parquet encontrado no Bronze.", {"RAW_DIR": RAW_DIR})

# Separar 24 tickers (.SA) para calendário e 31 (todos) para tabelão
# Critério: nome do arquivo (sem extensão) termina com '_sa' OU '.sa' (case-insensitive)
ticker_pat = re.compile(r"(_sa|\.sa)$", re.IGNORECASE)

ticker_files = [p for p in parquet_files if ticker_pat.search(os.path.splitext(os.path.basename(p))[0]) is not None]
other_files  = [p for p in parquet_files if p not in ticker_files]

# Precisamos de exatamente 24 tickers .SA
if len(ticker_files) != 24:
    fail("Quantidade de tickers .SA divergente para calendário.",
         {"esperado": 24, "encontrado": len(ticker_files),
          "amostra": [os.path.basename(x) for x in ticker_files][:10]})

# Precisamos de exatamente 31 artefatos no total (24 + 7)
if len(parquet_files) != 31:
    fail("Quantidade total de artefatos no Bronze divergente.",
         {"esperado": 31, "encontrado": len(parquet_files)})

# ============================== PASSO 2 ===============================
# Carregar os 24 tickers para construir o calendário (união dos índices)
calendar_indices = []
for fpath in ticker_files:
    _, ser = read_parquet_close_series(fpath)
    if not isinstance(ser.index, pd.DatetimeIndex):
        fail("Série sem DatetimeIndex ao construir calendário.", {"arquivo": fpath})
    calendar_indices.append(ser.index)

# União dos índices, ordenada, única e tz-free
if len(calendar_indices) == 0:
    fail("Nenhum índice coletado para calendário.", {})

cal_union = calendar_indices[0]
for idx in calendar_indices[1:]:
    cal_union = cal_union.union(idx)

# Normalizar resultados
cal_union = pd.DatetimeIndex(pd.to_datetime(cal_union, utc=False)).tz_localize(None).sort_values().unique()

# Validar contagem do calendário
if len(cal_union) != EXPECTED_ROWS:
    fail("contagem_pregoes_calendario_invalida",
         {"esperado": EXPECTED_ROWS, "obtido": int(len(cal_union))})

# Validar faixa de datas
if (cal_union.min() != pd.Timestamp(EXPECTED_DATE_MIN)) or (cal_union.max() != pd.Timestamp(EXPECTED_DATE_MAX)):
    fail("faixa_de_datas_calendario_invalida",
         {"esperado_min": EXPECTED_DATE_MIN, "obtido_min": str(cal_union.min().date()),
          "esperado_max": EXPECTED_DATE_MAX, "obtido_max": str(cal_union.max().date())})

# ============================== PASSO 3 ===============================
# Carregar TODOS os 31 ativos (24 tickers + 7 macros) e montar o tabelão por outer join no calendário
series_map = {}
for fpath in parquet_files:
    name, ser = read_parquet_close_series(fpath)
    if name in series_map:
        fail("Colisão de nomes de colunas derivadas de arquivos.", {"arquivo": fpath, "coluna": name})
    # Reindexar no calendário canônico (união), garantindo outer join controlado
    ser2 = ser.reindex(cal_union)
    series_map[name] = ser2

# Build do DataFrame final
df_silver_close = pd.DataFrame(index=cal_union, data={k: v for k, v in series_map.items()})

# ============================== PASSO 4 ===============================
# Validar estrutura em RAM: shape, datas, cardinalidade
shape_ok = (df_silver_close.shape == (EXPECTED_ROWS, EXPECTED_COLS))
if not shape_ok:
    fail("shape_regenerado_invalido",
         {"esperado": (EXPECTED_ROWS, EXPECTED_COLS), "obtido": df_silver_close.shape})

idx = df_silver_close.index
if (idx.min() != pd.Timestamp(EXPECTED_DATE_MIN)) or (idx.max() != pd.Timestamp(EXPECTED_DATE_MAX)):
    fail("faixa_de_datas_invalida",
         {"esperado_min": EXPECTED_DATE_MIN, "obtido_min": str(idx.min().date()),
          "esperado_max": EXPECTED_DATE_MAX, "obtido_max": str(idx.max().date())})

# ============================== PASSO 5 ===============================
# Calcular hash sha256 simulando persistência (buffer de bytes em memória)
buff = io.BytesIO()
try:
    df_silver_close.to_parquet(buff, index=True)
except Exception as e:
    fail("Falha ao serializar df_silver_close para Parquet em memória.", {"erro": repr(e)})
buff.seek(0)
sha256_hex = hashlib.sha256(buff.read()).hexdigest()

# ============================== PASSO 6 ===============================
# Atualizar manifesto em RAM: carregar CSV, localizar linha do ARTIFACT_NAME, inserir/atualizar 'sha256'
try:
    df_manifest = pd.read_csv(MANIFEST_CSV)
except Exception as e:
    fail("Falha ao ler o manifesto Silver.", {"MANIFEST_CSV": MANIFEST_CSV, "erro": repr(e)})

if "artifact_name" not in df_manifest.columns:
    fail("Manifesto inválido: coluna 'artifact_name' ausente.", {"colunas": list(df_manifest.columns)})

rows_match = df_manifest["artifact_name"] == EXPECTED_ARTIFACT_NAME
n_matches = int(rows_match.sum())
if n_matches != 1:
    fail("Manifesto inválido: linha do ARTIFACT_NAME não é única.",
         {"artifact_name": EXPECTED_ARTIFACT_NAME, "linhas_encontradas": n_matches})

# Inserir/atualizar coluna sha256 em RAM
df_manifest = df_manifest.copy()
if "sha256" not in df_manifest.columns:
    df_manifest["sha256"] = None
df_manifest.loc[rows_match, "sha256"] = sha256_hex

# ============================== PASSO 7 ===============================
# RELATÓRIO (DRY RUN) — nenhuma escrita em disco
print("\n====================== DRY RUN — RELATÓRIO ======================")
print("Tickers (.SA) carregados para calendário:", len(ticker_files), "(esperado=24)")
print("Calendário canônico (união de índices):", f"{idx.min().date()} → {idx.max().date()} (linhas={len(idx)})")
print("df_silver_close.shape:", df_silver_close.shape, " | esperado:", (EXPECTED_ROWS, EXPECTED_COLS))

print("\n--- df_silver_close.head() ---")
print(df_silver_close.head())

print("\n--- df_silver_close.info() ---")
buf_info = io.StringIO()
df_silver_close.info(buf=buf_info)
print(buf_info.getvalue())

print("\nsha256 do artefato regenerado (Parquet em memória):")
print(sha256_hex)

print("\n--- Linha do manifesto atualizada (em RAM) ---")
print(df_manifest.loc[rows_match])

# ============================== PASSO 8 ===============================
# CHECKLIST
checklist = {
    "24 tickers .SA carregados p/ calendário": len(ticker_files) == 24,
    "Calendário união == 3409 dias": len(idx) == EXPECTED_ROWS,
    "df_silver_close shape (3409,31)": df_silver_close.shape == (EXPECTED_ROWS, EXPECTED_COLS),
    "sha256 calculado exibido": isinstance(sha256_hex, str) and len(sha256_hex) == 64,
    "Manifesto carregado e linha atualizada em RAM": n_matches == 1 and df_manifest.loc[rows_match, "sha256"].iloc[0] == sha256_hex,
    "Nenhuma escrita em disco (DRY RUN)": DRY_RUN is True,
    "Relatório emitido": True,
}
print("\n--- CHECKLIST ---")
print(json.dumps(checklist, indent=2, ensure_ascii=False))

print("\nSTATUS_FINAL: OK (DRY RUN) — Nada foi persistido.")



=== VALIDATION_ERROR ===
Quantidade de tickers .SA divergente para calendário.
{
  "esperado": 24,
  "encontrado": 0,
  "amostra": []
}


SystemExit: 1